## Data

The `geoh5` format allows storing data (values) on different parts of an ``Object``. The `data_association` can be one of:

- OBJECT: Single element characterizing the parent object
- VERTEX: Array of values associated with the parent object vertices
- CELL: Array of values associated with the parent object cells 

Note: The length and order of the array provided must be consistent with the corresponding element of association.  

The data types supported by `geoh5py` are:

- Arrays
- Integer
- Text
- Color_map

![data](./images/data.png)

### Add data

Data can be added to an `Object` entity using the `add_data` method.

In [ ]:
import numpy as np

from geoh5py.objects import Curve
from geoh5py.workspace import Workspace


workspace = Workspace("my_project.geoh5")

# Create a straight Curve object
curve = Curve.create(
    workspace,       # The target Workspace
    name='FlightLine3',
    vertices=np.c_[np.linspace(0, 100, 100), np.zeros(100), np.zeros(100)]
)

# Add a single string comment
curve.add_data({
    "my_comment": {
        "association":"OBJECT", 
        "values": "hello_world"
    }
})

# Add a vector of floats
curve.add_data({
    "my_cell_values": {
        "association":"CELL", 
        "values": np.random.randn(curve.n_cells)
    }
})

# Add multiple data vectors on a single call
data = {}
for ii in range(8):
    data[f"Period:{ii}"] = {
        "association":"VERTEX", 
        "values": (ii+1) * np.cos(ii*curve.vertices[:, 0]*np.pi/curve.vertices[:, 0].max()/4.)
    }

data_list = curve.add_data(data)
print([obj.name for obj in data_list])

If the `association` argument is omited, `geoh5py` will attempt to assign the data to the correct part based on the shape of the data values, either `object.n_values` or `object.n_cells`

The newly created data is directly added to the project's `geoh5` file and available for visualization:

![adddata](./images/adddata.png)

### Get data
Just like any `Entity`, data can be retrieved from the `Workspace` using the `get_entity` method. For convenience, `Objects` also have a `get_data_list` and `get_data` method that focusses only on their respective children `Data`.

In [ ]:
my_list = curve.get_data_list()
print(my_list, curve.get_data(my_list[0]))

### Property Groups

`Data` entities sharing the same parent `Object` and `association` can be linked within a `property_groups` and made available through profiling. This can be used to group data that would normally be stored as 2D array.

In [ ]:
# Add another VERTEX data and create a group with previous
curve.add_data_to_group([obj.name for obj in data_list], "my_trig_group")

![propgroups](./images/propgroups.png)